<a href="https://colab.research.google.com/github/arjasc5231/moodots/blob/main/SER/CNN/CNNonMel_st5fold.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#SERwithCNNonMel_moredropout.ipynb의 모델로 5fold해봄
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
import numpy as np
import matplotlib.pyplot as plt
import os
import sklearn as sk
import sklearn.model_selection

In [17]:
learning_rate = 0.001
training_epochs = 61
batch_size = 100

In [4]:
mel_X_train, mel_X_test, mel_Y_train, mel_Y_test = np.load("/content/drive/MyDrive/team_runner/colab/dataset/emoDB/emo_mel_more.npy", allow_pickle=True)
mel_X = np.concatenate((mel_X_train, mel_X_test), axis=0)
mel_Y = np.concatenate((mel_Y_train, mel_Y_test), axis=0)

print(mel_X.shape)
print(mel_Y.shape)

(911, 128, 128)
(911,)


In [5]:
mel_X = np.expand_dims(mel_X, axis=-1)
mel_Y_origin = mel_Y[:]
mel_Y= to_categorical(mel_Y, 7)
print(mel_X.shape)
print(mel_Y_origin.shape)
print(mel_Y.shape)

(911, 128, 128, 1)
(911,)
(911, 7)


In [15]:
def create_model():
    model = keras.Sequential()
    model.add(keras.layers.Conv2D(filters=32, kernel_size=3, activation=tf.nn.relu, padding='SAME', 
                                  input_shape=(128, 128, 1)))
    model.add(keras.layers.MaxPool2D(padding='SAME'))
    model.add(keras.layers.Conv2D(filters=64, kernel_size=3, activation=tf.nn.relu, padding='SAME'))
    model.add(keras.layers.MaxPool2D(padding='SAME'))
    model.add(keras.layers.Conv2D(filters=128, kernel_size=3, activation=tf.nn.relu, padding='SAME'))
    model.add(keras.layers.MaxPool2D(padding='SAME'))
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(256, activation=tf.nn.relu))
    model.add(keras.layers.Dropout(0.5))
    model.add(keras.layers.Dense(128, activation=tf.nn.relu))
    model.add(keras.layers.Dropout(0.5))
    model.add(keras.layers.Dense(64, activation=tf.nn.relu))
    model.add(keras.layers.Dropout(0.5))
    model.add(keras.layers.Dense(7))
    return model

In [16]:
model = create_model()
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 128, 128, 32)      320       
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 64, 64, 64)        18496     
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 32, 32, 128)       73856     
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 16, 16, 128)       0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 32768)            

In [8]:
def loss_fn(model, images, labels):
    logits = model(images, training=True)
    loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(
        y_pred=logits, y_true=labels, from_logits=True))    
    return loss

In [9]:
def grad(model, images, labels):
    with tf.GradientTape() as tape:
        loss = loss_fn(model, images, labels)
    return tape.gradient(loss, model.variables)

In [10]:
def evaluate(model, images, labels):
    logits = model(images, training=False)
    correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(labels, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    return accuracy

In [18]:
print('Learning started. It takes sometime.')

skfold = sk.model_selection.StratifiedKFold(n_splits=5, shuffle=True)
fold = 1
fold_acc = []
for train_idx, test_idx in skfold.split(mel_X,mel_Y_origin): #kFold 생각나는대로 코딩한거라 비효율적일 수 있음. 인터넷에 검색해서 더 효율적인 코드로 수정하기
  model = create_model()
  optimizer = tf.optimizers.Adam(learning_rate=learning_rate)
  
  X_train, Y_train = mel_X[train_idx], mel_Y[train_idx]
  X_test, Y_test = mel_X[test_idx], mel_Y[test_idx]

  train_dataset = tf.data.Dataset.from_tensor_slices((X_train, Y_train)).batch(batch_size)
  test_dataset = tf.data.Dataset.from_tensor_slices((X_test, Y_test)).batch(batch_size)

  print(f'<<fold {fold}>>')

  for epoch in range(training_epochs):
    avg_loss = 0.
    avg_train_acc = 0.
    avg_test_acc = 0.
    train_step = 0
    test_step = 0

    for images, labels in train_dataset:
      grads = grad(model, images, labels)                
      optimizer.apply_gradients(zip(grads, model.variables))
      loss = loss_fn(model, images, labels)
      acc = evaluate(model, images, labels)
      avg_loss = avg_loss + loss
      avg_train_acc = avg_train_acc + acc
      train_step += 1
    avg_loss = avg_loss / train_step
    avg_train_acc = avg_train_acc / train_step
    
    conf_mat = [[0]*7 for i in range(7)]
    for images, labels in test_dataset:        
      acc = evaluate(model, images, labels)        
      avg_test_acc = avg_test_acc + acc
      test_step += 1
    avg_test_acc = avg_test_acc / test_step   

    if epoch%10==0: print('Epoch:', '{}'.format(epoch + 1), 'loss =', '{:.8f}'.format(avg_loss), 
          'train accuracy = ', '{:.4f}'.format(avg_train_acc), 
          'test accuracy = ', '{:.4f}'.format(avg_test_acc))
  
  fold_acc.append(avg_test_acc)
  fold+=1

  conf_mat = [[0]*7 for i in range(7)]
  for images, labels in test_dataset:
    logits = model(images, training=False)
    logits_max = tf.math.argmax(logits,1)
    labels_max = tf.math.argmax(labels,1)
    for i in range(len(logits)): conf_mat[logits_max[i]][labels_max[i]]+=1
  for i in range(7): print(conf_mat[i])
  conf_mat_normal = []
  for i in range(7):
    s = sum(conf_mat[i])
    if s==0: conf_mat_normal.append([0]*7)
    else: conf_mat_normal.append(list(map(lambda x:(x/s)*100, conf_mat[i])))
  label = ['anger','boredom','disgust','fear','happy','sad','neutral']
  print('\t'+'\t'.join(label))
  for i in range(7):
    print(label[i], end='')
    for j in range(7): print('\t%2.0f'%conf_mat_normal[i][j], end=' ')
    print()

print('Learning Finished!')
print('fold accuracy: ', fold_acc)
print('average accuracy: ', sum(fold_acc)/(fold-1))
#뒤로 갈수록 학습 성능이 떨어진다. 왜???

Learning started. It takes sometime.
<<fold 1>>
Epoch: 1 loss = 2.01811934 train accuracy =  0.2007 test accuracy =  0.2164
Epoch: 11 loss = 1.82735729 train accuracy =  0.3180 test accuracy =  0.3988
Epoch: 21 loss = 1.28746176 train accuracy =  0.5096 test accuracy =  0.4760
Epoch: 31 loss = 0.94543153 train accuracy =  0.7021 test accuracy =  0.5721
Epoch: 41 loss = 0.59647751 train accuracy =  0.8848 test accuracy =  0.6262
Epoch: 51 loss = 0.44707614 train accuracy =  0.9563 test accuracy =  0.6152
Epoch: 61 loss = 0.30584332 train accuracy =  0.9937 test accuracy =  0.6071
[31, 0, 1, 3, 8, 0, 0]
[1, 18, 1, 2, 0, 8, 4]
[1, 3, 11, 2, 1, 2, 1]
[3, 0, 4, 6, 0, 0, 1]
[4, 0, 0, 5, 9, 0, 2]
[0, 2, 0, 0, 0, 23, 0]
[0, 5, 2, 1, 4, 0, 14]
	anger	boredom	disgust	fear	happy	sad	neutral
anger	72 	 0 	 2 	 7 	19 	 0 	 0 
boredom	 3 	53 	 3 	 6 	 0 	24 	12 
disgust	 5 	14 	52 	10 	 5 	10 	 5 
fear	21 	 0 	29 	43 	 0 	 0 	 7 
happy	20 	 0 	 0 	25 	45 	 0 	10 
sad	 0 	 8 	 0 	 0 	 0 	92 	 0 
neut

In [ ]:
conf_mat = [[0]*7 for i in range(7)] #mat[real_label]=predicted_label list
print(conf_mat)
for images, labels in test_dataset:
  logits = model(images, training=False)
  logits_max = tf.math.argmax(logits,1)
  labels_max = tf.math.argmax(labels,1)
  for i in range(len(logits)): conf_mat[logits_max[i]][labels_max[i]]+=1

for i in range(7): print(conf_mat[i])

[[0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0]]


NameError: ignored

In [ ]:
conf_mat_normal = []
for i in range(7):
  s = sum(conf_mat[i])
  conf_mat_normal.append(list(map(lambda x:(x/s)*100, conf_mat[i])))

label = ['anger','boredom','disgust','fear','happy','sad','neutral']
print('\t'+'\t'.join(label))
for i in range(7):
  print(label[i], end='')
  for j in range(7): print('\t%2.0f'%conf_mat_normal[i][j], end=' ')
  print()